In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from matplotlib_venn import venn2
from matplotlib.gridspec import GridSpec
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================

# Paths
UP_DIR = Path('GO_enrichment_results')
DOWN_DIR = Path('GO_enrichment_results_downregulated')
OUTPUT_DIR = Path('publication_figures')
OUTPUT_DIR.mkdir(exist_ok=True)

# Parameters
TOP_N = 15
PADJ_THRESHOLD = 0.05

# Publication-ready style settings
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial', 'Helvetica']
plt.rcParams['font.size'] = 10
plt.rcParams['axes.linewidth'] = 1.2
plt.rcParams['xtick.major.width'] = 1.2
plt.rcParams['ytick.major.width'] = 1.2

# Color scheme (colorblind-friendly)
COLOR_UP = '#D55E00'      # Orange-red for upregulated
COLOR_DOWN = '#0072B2'    # Blue for downregulated
COLOR_SHARED = '#009E73'  # Green for shared
COLOR_NEUTRAL = '#999999' # Gray

print("="*70)
print("GENERATING PUBLICATION-READY FIGURES")
print("="*70)

# ============================================================================
# 1. LOAD DATA
# ============================================================================

print("\n[1/6] Loading data...")

import scanpy as sc
adata = sc.read('annotated.h5ad')
cell_types = adata.obs['Cell_Type'].unique().tolist()

# Load all enrichment results
upregulated_results = {}
downregulated_results = {}
up_gene_counts = {}
down_gene_counts = {}

for cell_type in cell_types:
    ct_clean = cell_type.replace(' ', '_')
    
    # Load upregulated
    up_file = UP_DIR / ct_clean / f'{ct_clean}_GO_enrichment.csv'
    if up_file.exists():
        df = pd.read_csv(up_file)
        upregulated_results[cell_type] = df[df['Adjusted P-value'] < PADJ_THRESHOLD].copy()
        
        # Get gene count from summary
        summary_file = UP_DIR / 'enrichment_summary.csv'
        if summary_file.exists():
            summary = pd.read_csv(summary_file)
            gene_count = summary[summary['Cell_Type'] == cell_type]['Upregulated_Genes'].values
            up_gene_counts[cell_type] = gene_count[0] if len(gene_count) > 0 else 0
        else:
            up_gene_counts[cell_type] = 0
    else:
        upregulated_results[cell_type] = pd.DataFrame()
        up_gene_counts[cell_type] = 0
    
    # Load downregulated
    down_file = DOWN_DIR / ct_clean / f'{ct_clean}_GO_enrichment.csv'
    if down_file.exists():
        df = pd.read_csv(down_file)
        downregulated_results[cell_type] = df[df['Adjusted P-value'] < PADJ_THRESHOLD].copy()
        
        # Get gene count
        summary_file = DOWN_DIR / 'enrichment_summary.csv'
        if summary_file.exists():
            summary = pd.read_csv(summary_file)
            gene_count = summary[summary['Cell_Type'] == cell_type]['Downregulated_Genes'].values
            down_gene_counts[cell_type] = gene_count[0] if len(gene_count) > 0 else 0
        else:
            down_gene_counts[cell_type] = 0
    else:
        downregulated_results[cell_type] = pd.DataFrame()
        down_gene_counts[cell_type] = 0

print(f"  ✓ Loaded enrichment results for {len(cell_types)} cell types")

# Combine all results
all_up_terms = []
all_down_terms = []

for cell_type in cell_types:
    if len(upregulated_results[cell_type]) > 0:
        up_df = upregulated_results[cell_type].copy()
        up_df['Cell_Type'] = cell_type
        all_up_terms.append(up_df)
    
    if len(downregulated_results[cell_type]) > 0:
        down_df = downregulated_results[cell_type].copy()
        down_df['Cell_Type'] = cell_type
        all_down_terms.append(down_df)

all_up_df = pd.concat(all_up_terms, ignore_index=True) if all_up_terms else pd.DataFrame()
all_down_df = pd.concat(all_down_terms, ignore_index=True) if all_down_terms else pd.DataFrame()

# ============================================================================
# 2. FIGURE 1: MULTI-PANEL OVERVIEW (HEATMAPS + BAR CHARTS)
# ============================================================================

print("\n[2/6] Creating Figure 1: Multi-panel overview...")

fig = plt.figure(figsize=(16, 10))
gs = GridSpec(2, 3, figure=fig, hspace=0.3, wspace=0.3)

# Panel A: DEG counts per cell type
ax1 = fig.add_subplot(gs[0, 0])
ct_sorted = sorted(cell_types, key=lambda x: up_gene_counts[x] + down_gene_counts[x], reverse=True)
y_pos = np.arange(len(ct_sorted))

up_counts = [up_gene_counts[ct] for ct in ct_sorted]
down_counts = [down_gene_counts[ct] for ct in ct_sorted]

ax1.barh(y_pos, up_counts, height=0.4, label='Upregulated', color=COLOR_UP, alpha=0.8)
ax1.barh(y_pos - 0.4, down_counts, height=0.4, label='Downregulated', color=COLOR_DOWN, alpha=0.8)
ax1.set_yticks(y_pos - 0.2)
ax1.set_yticklabels(ct_sorted, fontsize=8)
ax1.set_xlabel('Number of DEGs', fontsize=10, fontweight='bold')
ax1.set_title('A. DEG Counts per Cell Type', fontsize=11, fontweight='bold', loc='left')
ax1.legend(fontsize=8)
ax1.grid(True, alpha=0.3, axis='x')
ax1.invert_yaxis()

# Panel B: Enriched pathway counts
ax2 = fig.add_subplot(gs[0, 1])
up_term_counts = [len(upregulated_results[ct]) for ct in ct_sorted]
down_term_counts = [len(downregulated_results[ct]) for ct in ct_sorted]

ax2.barh(y_pos, up_term_counts, height=0.4, label='Upregulated', color=COLOR_UP, alpha=0.8)
ax2.barh(y_pos - 0.4, down_term_counts, height=0.4, label='Downregulated', color=COLOR_DOWN, alpha=0.8)
ax2.set_yticks(y_pos - 0.2)
ax2.set_yticklabels(ct_sorted, fontsize=8)
ax2.set_xlabel('Number of Enriched Pathways', fontsize=10, fontweight='bold')
ax2.set_title('B. Enriched Pathway Counts', fontsize=11, fontweight='bold', loc='left')
ax2.legend(fontsize=8)
ax2.grid(True, alpha=0.3, axis='x')
ax2.invert_yaxis()

# Panel C: Top upregulated pathways
ax3 = fig.add_subplot(gs[0, 2])
if len(all_up_df) > 0:
    top_up = all_up_df['Term'].value_counts().head(10)
    ax3.barh(range(len(top_up)), top_up.values, color=COLOR_UP, alpha=0.8)
    ax3.set_yticks(range(len(top_up)))
    ax3.set_yticklabels([t[:40] + '...' if len(t) > 40 else t for t in top_up.index], fontsize=8)
    ax3.set_xlabel('Cell Type Count', fontsize=10, fontweight='bold')
    ax3.set_title('C. Top Upregulated Pathways', fontsize=11, fontweight='bold', loc='left')
    ax3.grid(True, alpha=0.3, axis='x')
    ax3.invert_yaxis()

# Panel D: Top downregulated pathways
ax4 = fig.add_subplot(gs[1, 0])
if len(all_down_df) > 0:
    top_down = all_down_df['Term'].value_counts().head(10)
    ax4.barh(range(len(top_down)), top_down.values, color=COLOR_DOWN, alpha=0.8)
    ax4.set_yticks(range(len(top_down)))
    ax4.set_yticklabels([t[:40] + '...' if len(t) > 40 else t for t in top_down.index], fontsize=8)
    ax4.set_xlabel('Cell Type Count', fontsize=10, fontweight='bold')
    ax4.set_title('D. Top Downregulated Pathways', fontsize=11, fontweight='bold', loc='left')
    ax4.grid(True, alpha=0.3, axis='x')
    ax4.invert_yaxis()

# Panel E: Pathway overlap
ax5 = fig.add_subplot(gs[1, 1])
up_unique = set(all_up_df['Term'].unique()) if len(all_up_df) > 0 else set()
down_unique = set(all_down_df['Term'].unique()) if len(all_down_df) > 0 else set()

if len(up_unique) > 0 and len(down_unique) > 0:
    venn2([up_unique, down_unique],
          set_labels=('Up', 'Down'),
          set_colors=(COLOR_UP, COLOR_DOWN),
          alpha=0.6,
          ax=ax5)
    ax5.set_title('E. Pathway Term Overlap', fontsize=11, fontweight='bold', loc='left')

# Panel F: Database distribution
ax6 = fig.add_subplot(gs[1, 2])
if len(all_up_df) > 0 or len(all_down_df) > 0:
    databases = []
    counts_up = []
    counts_down = []
    
    for db in ['GO_Biological_Process', 'KEGG']:
        databases.append(db.replace('_', ' '))
        
        if len(all_up_df) > 0:
            counts_up.append(len(all_up_df[all_up_df['Gene_set'].str.contains(db, na=False)]))
        else:
            counts_up.append(0)
        
        if len(all_down_df) > 0:
            counts_down.append(len(all_down_df[all_down_df['Gene_set'].str.contains(db, na=False)]))
        else:
            counts_down.append(0)
    
    x = np.arange(len(databases))
    width = 0.35
    
    ax6.bar(x - width/2, counts_up, width, label='Upregulated', color=COLOR_UP, alpha=0.8)
    ax6.bar(x + width/2, counts_down, width, label='Downregulated', color=COLOR_DOWN, alpha=0.8)
    ax6.set_xticks(x)
    ax6.set_xticklabels(databases, rotation=15, ha='right', fontsize=9)
    ax6.set_ylabel('Number of Terms', fontsize=10, fontweight='bold')
    ax6.set_title('F. Database Distribution', fontsize=11, fontweight='bold', loc='left')
    ax6.legend(fontsize=8)
    ax6.grid(True, alpha=0.3, axis='y')

plt.savefig(OUTPUT_DIR / 'Figure1_Overview.png', dpi=300, bbox_inches='tight')
plt.savefig(OUTPUT_DIR / 'Figure1_Overview.pdf', bbox_inches='tight')
plt.close()
print("  ✓ Figure 1 saved (PNG + PDF)")

# ============================================================================
# 3. FIGURE 2: TOP PATHWAYS BUBBLE PLOT
# ============================================================================

print("\n[3/6] Creating Figure 2: Top pathways bubble plot...")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 10))

# Upregulated bubble plot
if len(all_up_df) > 0:
    # Get top terms
    top_terms = all_up_df['Term'].value_counts().head(TOP_N).index
    
    # Create data for bubble plot
    bubble_data = []
    for term in top_terms:
        term_data = all_up_df[all_up_df['Term'] == term]
        for _, row in term_data.iterrows():
            gene_count = int(row['Overlap'].split('/')[0])
            bubble_data.append({
                'Term': term,
                'Cell_Type': row['Cell_Type'],
                'Adj_Pval': row['Adjusted P-value'],
                'Gene_Count': gene_count,
                'Combined_Score': row['Combined Score']
            })
    
    bubble_df = pd.DataFrame(bubble_data)
    
    # Create pivot for heatmap background
    pivot = bubble_df.pivot_table(
        index='Term',
        columns='Cell_Type',
        values='Adj_Pval',
        fill_value=1
    )
    
    # Plot
    for i, term in enumerate(pivot.index):
        for j, ct in enumerate(pivot.columns):
            pval = pivot.loc[term, ct]
            if pval < PADJ_THRESHOLD:
                row_data = bubble_df[(bubble_df['Term'] == term) & (bubble_df['Cell_Type'] == ct)]
                if len(row_data) > 0:
                    size = row_data.iloc[0]['Gene_Count'] * 20
                    color_val = -np.log10(pval)
                    ax1.scatter(j, i, s=size, c=[color_val], cmap='Reds', 
                              vmin=0, vmax=10, alpha=0.7, edgecolors='black', linewidth=0.5)
    
    ax1.set_xticks(range(len(pivot.columns)))
    ax1.set_xticklabels(pivot.columns, rotation=45, ha='right', fontsize=8)
    ax1.set_yticks(range(len(pivot.index)))
    ax1.set_yticklabels([t[:50] + '...' if len(t) > 50 else t for t in pivot.index], fontsize=8)
    ax1.set_title(f'Upregulated Pathways\n(Top {TOP_N}, size=gene count)', 
                 fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.2)
    
    # Colorbar for upregulated
    sm = plt.cm.ScalarMappable(cmap='Reds', norm=plt.Normalize(vmin=0, vmax=10))
    sm.set_array([])
    cbar1 = plt.colorbar(sm, ax=ax1)
    cbar1.set_label('-log10(Adj P-value)', fontsize=9)

# Downregulated bubble plot
if len(all_down_df) > 0:
    top_terms = all_down_df['Term'].value_counts().head(TOP_N).index
    
    bubble_data = []
    for term in top_terms:
        term_data = all_down_df[all_down_df['Term'] == term]
        for _, row in term_data.iterrows():
            gene_count = int(row['Overlap'].split('/')[0])
            bubble_data.append({
                'Term': term,
                'Cell_Type': row['Cell_Type'],
                'Adj_Pval': row['Adjusted P-value'],
                'Gene_Count': gene_count,
                'Combined_Score': row['Combined Score']
            })
    
    bubble_df = pd.DataFrame(bubble_data)
    
    pivot = bubble_df.pivot_table(
        index='Term',
        columns='Cell_Type',
        values='Adj_Pval',
        fill_value=1
    )
    
    for i, term in enumerate(pivot.index):
        for j, ct in enumerate(pivot.columns):
            pval = pivot.loc[term, ct]
            if pval < PADJ_THRESHOLD:
                row_data = bubble_df[(bubble_df['Term'] == term) & (bubble_df['Cell_Type'] == ct)]
                if len(row_data) > 0:
                    size = row_data.iloc[0]['Gene_Count'] * 20
                    color_val = -np.log10(pval)
                    ax2.scatter(j, i, s=size, c=[color_val], cmap='Blues',
                              vmin=0, vmax=10, alpha=0.7, edgecolors='black', linewidth=0.5)
    
    ax2.set_xticks(range(len(pivot.columns)))
    ax2.set_xticklabels(pivot.columns, rotation=45, ha='right', fontsize=8)
    ax2.set_yticks(range(len(pivot.index)))
    ax2.set_yticklabels([t[:50] + '...' if len(t) > 50 else t for t in pivot.index], fontsize=8)
    ax2.set_title(f'Downregulated Pathways\n(Top {TOP_N}, size=gene count)', 
                 fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.2)
    
    sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=0, vmax=10))
    sm.set_array([])
    cbar2 = plt.colorbar(sm, ax=ax2)
    cbar2.set_label('-log10(Adj P-value)', fontsize=9)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'Figure2_BubblePlot.png', dpi=300, bbox_inches='tight')
plt.savefig(OUTPUT_DIR / 'Figure2_BubblePlot.pdf', bbox_inches='tight')
plt.close()
print("  ✓ Figure 2 saved (PNG + PDF)")

# ============================================================================
# 4. FIGURE 3: CELL TYPE COMPARISON
# ============================================================================

print("\n[4/6] Creating Figure 3: Cell type comparison...")

# Select top 3 cell types by total pathway count
ct_pathway_counts = []
for ct in cell_types:
    total = len(upregulated_results[ct]) + len(downregulated_results[ct])
    ct_pathway_counts.append((ct, total))

top_celltypes = sorted(ct_pathway_counts, key=lambda x: x[1], reverse=True)[:3]
top_ct_names = [ct[0] for ct in top_celltypes]

fig = plt.figure(figsize=(16, 12))
gs = GridSpec(3, 2, figure=fig, hspace=0.3, wspace=0.3)

for idx, cell_type in enumerate(top_ct_names):
    # Upregulated panel
    ax_up = fig.add_subplot(gs[idx, 0])
    up_df = upregulated_results[cell_type]
    
    if len(up_df) > 0:
        top_up = up_df.sort_values('Adjusted P-value').head(10)
        
        y_pos = range(len(top_up))
        bars = ax_up.barh(y_pos, -np.log10(top_up['Adjusted P-value']), color=COLOR_UP, alpha=0.8)
        
        # Add gene counts as text
        for i, (_, row) in enumerate(top_up.iterrows()):
            gene_count = row['Overlap'].split('/')[0]
            ax_up.text(-np.log10(row['Adjusted P-value']) + 0.3, i, f"n={gene_count}",
                      va='center', fontsize=7)
        
        ax_up.set_yticks(y_pos)
        ax_up.set_yticklabels([t[:45] + '...' if len(t) > 45 else t for t in top_up['Term']], fontsize=8)
        ax_up.set_xlabel('-log10(Adjusted P-value)', fontsize=9, fontweight='bold')
        ax_up.set_title(f'{cell_type} - Upregulated\n(n={up_gene_counts[cell_type]} genes)', 
                       fontsize=10, fontweight='bold')
        ax_up.axvline(-np.log10(0.05), color='red', linestyle='--', linewidth=1, alpha=0.5)
        ax_up.grid(True, alpha=0.3, axis='x')
        ax_up.invert_yaxis()
    
    # Downregulated panel
    ax_down = fig.add_subplot(gs[idx, 1])
    down_df = downregulated_results[cell_type]
    
    if len(down_df) > 0:
        top_down = down_df.sort_values('Adjusted P-value').head(10)
        
        y_pos = range(len(top_down))
        bars = ax_down.barh(y_pos, -np.log10(top_down['Adjusted P-value']), color=COLOR_DOWN, alpha=0.8)
        
        for i, (_, row) in enumerate(top_down.iterrows()):
            gene_count = row['Overlap'].split('/')[0]
            ax_down.text(-np.log10(row['Adjusted P-value']) + 0.3, i, f"n={gene_count}",
                        va='center', fontsize=7)
        
        ax_down.set_yticks(y_pos)
        ax_down.set_yticklabels([t[:45] + '...' if len(t) > 45 else t for t in top_down['Term']], fontsize=8)
        ax_down.set_xlabel('-log10(Adjusted P-value)', fontsize=9, fontweight='bold')
        ax_down.set_title(f'{cell_type} - Downregulated\n(n={down_gene_counts[cell_type]} genes)', 
                         fontsize=10, fontweight='bold')
        ax_down.axvline(-np.log10(0.05), color='red', linestyle='--', linewidth=1, alpha=0.5)
        ax_down.grid(True, alpha=0.3, axis='x')
        ax_down.invert_yaxis()

plt.savefig(OUTPUT_DIR / 'Figure3_CellTypeComparison.png', dpi=300, bbox_inches='tight')
plt.savefig(OUTPUT_DIR / 'Figure3_CellTypeComparison.pdf', bbox_inches='tight')
plt.close()
print("  ✓ Figure 3 saved (PNG + PDF)")

# ============================================================================
# 5. FIGURE 4: PATHWAY CATEGORIES GROUPED
# ============================================================================

print("\n[5/6] Creating Figure 4: Pathway categories...")

# Define pathway categories (simplified functional grouping)
categories = {
    'Cell Cycle & Proliferation': ['cell cycle', 'mitosis', 'DNA replication', 'cell division', 'proliferation'],
    'Immune Response': ['immune', 'inflammation', 'cytokine', 'chemokine', 'interferon', 'T cell', 'B cell'],
    'Metabolism': ['metabolic', 'metabolism', 'glycolysis', 'oxidative phosphorylation', 'lipid', 'glucose'],
    'Cell Adhesion & Migration': ['adhesion', 'migration', 'motility', 'ECM', 'extracellular matrix'],
    'Signal Transduction': ['signaling', 'signal transduction', 'pathway', 'MAPK', 'WNT', 'receptor'],
    'Apoptosis & Cell Death': ['apoptosis', 'cell death', 'programmed cell death'],
    'Differentiation': ['differentiation', 'development', 'morphogenesis']
}

def categorize_term(term):
    """Assign term to category"""
    term_lower = term.lower()
    for category, keywords in categories.items():
        if any(keyword in term_lower for keyword in keywords):
            return category
    return 'Other'

# Categorize all terms
if len(all_up_df) > 0:
    all_up_df['Category'] = all_up_df['Term'].apply(categorize_term)

if len(all_down_df) > 0:
    all_down_df['Category'] = all_down_df['Term'].apply(categorize_term)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Upregulated categories
if len(all_up_df) > 0:
    cat_counts_up = all_up_df['Category'].value_counts()
    ax1.barh(range(len(cat_counts_up)), cat_counts_up.values, color=COLOR_UP, alpha=0.8)
    ax1.set_yticks(range(len(cat_counts_up)))
    ax1.set_yticklabels(cat_counts_up.index, fontsize=10)
    ax1.set_xlabel('Number of Enriched Terms', fontsize=11, fontweight='bold')
    ax1.set_title('Upregulated Pathway Categories', fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3, axis='x')
    ax1.invert_yaxis()

# Downregulated categories
if len(all_down_df) > 0:
    cat_counts_down = all_down_df['Category'].value_counts()
    ax2.barh(range(len(cat_counts_down)), cat_counts_down.values, color=COLOR_DOWN, alpha=0.8)
    ax2.set_yticks(range(len(cat_counts_down)))
    ax2.set_yticklabels(cat_counts_down.index, fontsize=10)
    ax2.set_xlabel('Number of Enriched Terms', fontsize=11, fontweight='bold')
    ax2.set_title('Downregulated Pathway Categories', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='x')
    ax2.invert_yaxis()

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'Figure4_PathwayCategories.png', dpi=300, bbox_inches='tight')
plt.savefig(OUTPUT_DIR / 'Figure4_PathwayCategories.pdf', bbox_inches='tight')
plt.close()
print("  ✓ Figure 4 saved (PNG + PDF)")

# ============================================================================
# 6. FIGURE 5: VENN DIAGRAMS FOR KEY CELL TYPES
# ============================================================================

print("\n[6/6] Creating Figure 5: Venn diagrams...")

# Use top 6 cell types
top_6_celltypes = [ct[0] for ct in sorted(ct_pathway_counts, key=lambda x: x[1], reverse=True)[:6]]

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for idx, cell_type in enumerate(top_6_celltypes):
    ax = axes[idx]
    
    up_terms = set(upregulated_results[cell_type]['Term'].unique()) if len(upregulated_results[cell_type]) > 0 else set()
    down_terms = set(downregulated_results[cell_type]['Term'].unique()) if len(downregulated_results[cell_type]) > 0 else set()
    
    if len(up_terms) > 0 or len(down_terms) > 0:
        venn2([up_terms, down_terms],
              set_labels=('Up', 'Down'),
              set_colors=(COLOR_UP, COLOR_DOWN),
              alpha=0.6,
              ax=ax)
        
        total_up = len(upregulated_results[cell_type])
        total_down = len(downregulated_results[cell_type])
        
        ax.set_title(f'{cell_type}\n({total_up} up, {total_down} down pathways)', 
                    fontsize=9, fontweight='bold')
    else:
        ax.text(0.5, 0.5, 'No enriched pathways', ha='center', va='center', transform=ax.transAxes)
        ax.set_title(cell_type, fontsize=9, fontweight='bold')
        ax.axis('off')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'Figure5_VennDiagrams.png', dpi=300, bbox_inches='tight')
plt.savefig(OUTPUT_DIR / 'Figure5_VennDiagrams.pdf', bbox_inches='tight')
plt.close()
print("  ✓ Figure 5 saved (PNG + PDF)")

# ============================================================================
# 7. CREATE SUPPLEMENTARY TABLE
# ============================================================================

print("\n[7/7] Creating supplementary table...")

supp_data = []
for cell_type in cell_types:
    supp_data.append({
        'Cell_Type': cell_type,
        'Upregulated_Genes': up_gene_counts[cell_type],
        'Downregulated_Genes': down_gene_counts[cell_type],
        'Up_Pathways': len(upregulated_results[cell_type]),
        'Down_Pathways': len(downregulated_results[cell_type]),
        'Up_GO': len(upregulated_results[cell_type][upregulated_results[cell_type]['Gene_set'].str.contains('GO', na=False)]) if len(upregulated_results[cell_type]) > 0 else 0,
        'Up_KEGG': len(upregulated_results[cell_type][upregulated_results[cell_type]['Gene_set'].str.contains('KEGG', na=False)]) if len(upregulated_results[cell_type]) > 0 else 0,
        'Down_GO': len(downregulated_results[cell_type][downregulated_results[cell_type]['Gene_set'].str.contains('GO', na=False)]) if len(downregulated_results[cell_type]) > 0 else 0,
        'Down_KEGG': len(downregulated_results[cell_type][downregulated_results[cell_type]['Gene_set'].str.contains('KEGG', na=False)]) if len(downregulated_results[cell_type]) > 0 else 0
    })

supp_table = pd.DataFrame(supp_data)
supp_table = supp_table.sort_values('Upregulated_Genes', ascending=False)
supp_table.to_csv(OUTPUT_DIR / 'Supplementary_Table_GO_Summary.csv', index=False)
print("  ✓ Supplementary table saved")

# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("\n" + "="*70)
print("PUBLICATION FIGURES COMPLETE!")
print("="*70)
print(f"\n📊 Generated Figures:")
print(f"  • Figure 1: Multi-panel overview (heatmaps + bar charts)")
print(f"  • Figure 2: Top pathways bubble plot")
print(f"  • Figure 3: Cell type comparison (top 3 cell types)")
print(f"  • Figure 4: Pathway categories grouped")
print(f"  • Figure 5: Venn diagrams (top 6 cell types)")
print(f"  • Supplementary Table: Complete GO summary")
print(f"\n📁 Output Location: {OUTPUT_DIR}")
print(f"  • All figures saved in both PNG (300 DPI) and PDF formats")
print(f"  • Ready for publication submission")
print("\n💡 Usage Tips:")
print("  • Use PNG for presentations and online submission")
print("  • Use PDF for final print publication (vector format)")
print("  • Edit text/labels in PDF using Adobe Illustrator if needed")
print("  • All figures use colorblind-friendly palettes")
print("\n✅ All publication-ready figures generated successfully!")
print("="*70)

GENERATING PUBLICATION-READY FIGURES

[1/6] Loading data...


findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the foll

  ✓ Loaded enrichment results for 21 cell types

[2/6] Creating Figure 1: Multi-panel overview...


findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the foll

  ✓ Figure 1 saved (PNG + PDF)

[3/6] Creating Figure 2: Top pathways bubble plot...


...

  ✓ Figure 3 saved (PNG + PDF)

[5/6] Creating Figure 4: Pathway categories...


findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the foll

  ✓ Figure 4 saved (PNG + PDF)

[6/6] Creating Figure 5: Venn diagrams...


findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the foll

  ✓ Figure 5 saved (PNG + PDF)

[7/7] Creating supplementary table...
  ✓ Supplementary table saved

PUBLICATION FIGURES COMPLETE!

📊 Generated Figures:
  • Figure 1: Multi-panel overview (heatmaps + bar charts)
  • Figure 2: Top pathways bubble plot
  • Figure 3: Cell type comparison (top 3 cell types)
  • Figure 4: Pathway categories grouped
  • Figure 5: Venn diagrams (top 6 cell types)
  • Supplementary Table: Complete GO summary

📁 Output Location: publication_figures
  • All figures saved in both PNG (300 DPI) and PDF formats
  • Ready for publication submission

💡 Usage Tips:
  • Use PNG for presentations and online submission
  • Use PDF for final print publication (vector format)
  • Edit text/labels in PDF using Adobe Illustrator if needed
  • All figures use colorblind-friendly palettes

✅ All publication-ready figures generated successfully!
